In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from pathlib import Path
import sys
import geopandas as gpd
import pandas as pd


scripts_dir = Path("../..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())
from helpers.mapshaper import Mapshaper
from helpers.tippcanoe import mbtileGeneration
from helpers.mapbox_uploader import uploadToMapbox
from helpers.settings import get_settings
from helpers.file_handler import FileConventionHandler
from helpers.utils import download_and_unzip_if_needed, writeReadGCP

from data_commons.loader import load_regions

In [3]:
mysettings = get_settings()
prev_step = "preprocess"
current_step = "tiles"

### EEZs: Exclusive Economic Zones 

In [5]:
pipe = "eez"
collection_name = f"{pipe}_v11"

eez_dir = FileConventionHandler(pipe)
# Download the EEZ file && unzip it
download_and_unzip_if_needed(eez_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(8).input([eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields=",".join(["GEONAME", "POL_TYPE", "ISO_SOV1", "ISO_SOV2", "ISO_SOV3"])
).output(
    eez_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()
mbtileGeneration(
    eez_dir.get_step_fmt_file_path(current_step, "json"),
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
)

/home/mambauser/data/eez/processed/eez_preprocess.zip
/home/mambauser/data/eez/processed/preprocess


Allocating 8 GB of heap memory
[o] Wrote /home/mambauser/data/eez/processed/tiles/eez_tiles.json


PosixPath('/home/mambauser/data/eez/processed/tiles/eez_v11.mbtiles')

In [ ]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

#### EEZs: wdpa Regions

In [21]:
collection_name = "regions"

# load the EEZ file & the regions file
eez_data = gpd.read_file(eez_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())
regions_df = pd.DataFrame(
    [
        {"region_id": data["region_iso"], "location_id": iso}
        for data in load_regions().get("data", [])
        for iso in data["country_iso_3s"]
    ]
)

# merge the two files
gpd.GeoDataFrame(
    pd.merge(
        eez_data,
        regions_df,
        how="left",
        left_on="ISO_SOV1",
        right_on="location_id",
        sort=True,
        copy=True,
    ),
    crs=eez_data.crs,
).to_file(
    filename=eez_dir.get_processed_step_path(prev_step)
    .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
    .as_posix(),
    driver="ESRI Shapefile",
)

# dissolve by region_id

Mapshaper(16).input(
    [
        eez_dir.get_processed_step_path(prev_step)
        .joinpath(f"{pipe}_{prev_step}_{collection_name}.shp")
        .as_posix()
    ]
).dissolve2(fields="region_id").output(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json").as_posix(),
    force=True,
    format="geojson",
).execute()

# generate the mbtiles
mbtileGeneration(eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.json"))

NameError: name 'eez_dir' is not defined

In [34]:
uploadToMapbox(
    eez_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/eez/processed/tiles/regions.mbtiles to s3://tilestream-tilesets-production/97/_pending/ojc7oxn5cpu10yo0o9tsl1xlc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:00<00:00,  1.81s/it]


True

### Countries - gadm

In [6]:
pipe = "gadm"
collection_name = f"{pipe}_simplified"

gadm_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(gadm_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(64).input([gadm_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields=",".join(["GID_0", "COUNTRY", "COUNTRY_ES", "COUNTRY_FR", 'area_km2'])
).output(
    gadm_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# Generate the mbtiles
mbtileGeneration(
    gadm_dir.get_step_fmt_file_path(current_step, "json"),
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    True
)

/home/mambauser/data/gadm/processed/gadm_preprocess.zip
/home/mambauser/data/gadm/processed/preprocess


Allocating 64 GB of heap memory
[o] Wrote /home/mambauser/data/gadm/processed/tiles/gadm_tiles.json
For layer 0, using name "gadm_tiles"
/home/mambauser/data/gadm/processed/tiles/gadm_tiles.json:12: Found ] at top level
/home/mambauser/data/gadm/processed/tiles/gadm_tiles.json:10: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
206 features, 181286774 bytes of geometry, 3347 bytes of separate metadata, 9901 bytes of string pool
Choosing a maxzoom of -z0 for features about 2772178 feet (844960 meters) apart
Choosing a maxzoom of -z10 for resolution of about 317 feet (96 meters) within features
  99.9%  10/128/306   


PosixPath('/home/mambauser/data/gadm/processed/tiles/gadm_simplified.mbtiles')

In [9]:
uploadToMapbox(
    gadm_dir.get_processed_step_path(current_step).joinpath(f"{collection_name}.mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/gadm/processed/tiles/gadm_simplified.mbtiles to s3://tilestream-tilesets-production/69/_pending/6o7ulrkerlm1xi20i3scyy0mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [03:43<00:00,  2.23s/it]


True

### MPAs: Marine Protected Areas from WDPA

In [9]:
pipe = "mpa"
collection_name = "mpas_wdpa"

source_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# simplify the geometries
Mapshaper(16).input([source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()]).filter_fields(
    fields="WDPAID,NAME,PA_DEF,GIS_M_AREA,PARENT_ISO"
).clean(allow_overlaps=True, rewind=True).simplify("dp 10% keep-shapes planar").clean(
    allow_overlaps=True
).output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

/home/mambauser/data/mpa/processed/mpa_preprocess.zip
/home/mambauser/data/mpa/processed/preprocess


Allocating 16 GB of heap memory
[clean] Retained 19,184 of 19,184 features
[simplify] Repaired 277,407 intersections; 443,678 intersections could not be repaired
[clean] Retained 19,184 of 19,184 features
[o] Wrote /home/mambauser/data/mpa/processed/tiles/mpa_tiles.json


PosixPath('/home/mambauser/data/mpa/processed/tiles/mpa_tiles.mbtiles')

In [10]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpa/processed/tiles/mpa_tiles.mbtiles to s3://tilestream-tilesets-production/de/_pending/yvng0dxxxru12eq9ye80350mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


True

### PAs: Terrestrial Protected Areas from WDPA

In [30]:
def split_n_parts(gdf: gpd.GeoDataFrame, folder: Path, n:int) -> None:
    
    for i in range(n):
        path = folder.joinpath(f"part{i}.shp")
        gdf.iloc[i * len(gdf) // n : (i + 1) * len(gdf) // n].to_file(path, driver="ESRI Shapefile")

In [31]:
import asyncio
from pipelines.utils import background
from pipelines.processors import repair_geometry

from tqdm.asyncio import tqdm


@background
def simplify(geometry, pbar, tlrc=0.0001) -> gpd.GeoDataFrame:
    try:
        return repair_geometry(geometry.simplify(tlrc))
    except Exception as e:
        print(e)
        return geometry
    finally:
        pbar.update(1)


async def simplify_async(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    with tqdm(total=gdf.shape[0]) as pbar:
        gdf["geometry"] = await asyncio.gather(
            *(simplify(val, pbar) for val in gdf["geometry"])
        )
    return gdf

In [32]:
pipe = "mpa-terrestrial"
collection_name = "pas_wdpa"

source_dir = FileConventionHandler(pipe)

In [33]:
# Download the file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings, "gpkg")

# split the file in two parts
file = gpd.read_file(source_dir.get_step_fmt_file_path(prev_step, "gpkg").as_posix())

/home/mambauser/data/mpa-terrestrial/processed/mpa-terrestrial_preprocess.gpkg
/home/mambauser/data/mpa-terrestrial/processed/preprocess


In [34]:
test = await simplify_async(file)

100%|██████████| 292261/292261 [03:19<00:00, 1461.41it/s]


In [46]:
test.to_file(source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix(), driver="ESRI Shapefile")

In [45]:
source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix()

'/home/mambauser/data/mpa-terrestrial/processed/preprocess/mpa-terrestrial_preprocess.shp'

In [10]:
i_folder = source_dir.get_processed_step_path(prev_step).joinpath("parts")
i_folder.mkdir(exist_ok=True, parents=True)
folders = split_n_parts(test, i_folder, 100)
del test
del file

In [22]:
Mapshaper(32).input(
    [f"{i_folder}/*.shp"],
).filter_fields(
    fields="WDPAID,NAME,PA_DEF,GIS_AREA,PARENT_ISO"
).clean(allow_overlaps=True, rewind=True).simplify("dp 10% keep-shapes planar").clean(
    allow_overlaps=True
).output(
    source_dir.get_processed_step_path(current_step), force=True, format="geojson"
).debug()

mapshaper-xl 32gb -i /home/mambauser/data/mpa-terrestrial/processed/preprocess/parts/*.shp  -filter-fields fields=WDPAID,NAME,PA_DEF,GIS_AREA,PARENT_ISO -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/mpa-terrestrial/processed/tiles force format=geojson


In [29]:
!mapshaper-xl 32gb -i /home/mambauser/data/mpa-terrestrial/processed/tiles/part*.json combine-files -merge-layers -o /home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.json force format=geojson

Allocating 32 GB of heap memory
[o] RFC 7946 warning: non-WGS84 GeoJSON output.
[o] Wrote /home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.json


In [28]:
!mapshaper-xl 32gb -i /home/mambauser/data/mpa-terrestrial/processed/preprocess/parts/*.shp  -filter-fields fields=WDPAID,NAME,PA_DEF,GIS_AREA,PARENT_ISO -clean allow-overlaps rewind -simplify dp 30% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/mpa-terrestrial/processed/tiles force combine-layers format=geojson


Allocating 32 GB of heap memory
[clean] Retained 2,922 of 2,922 features
[simplify] Repaired 14,231 intersections; 128 intersections could not be repaired
[clean] Retained 2,922 of 2,922 features
[o] Wrote /home/mambauser/data/mpa-terrestrial/processed/tiles/part0.json
Allocating 32 GB of heap memory
[clean] Retained 2,923 of 2,923 features
[simplify] Repaired 2,300 intersections; 253 intersections could not be repaired
[clean] Retained 2,923 of 2,923 features
[o] Wrote /home/mambauser/data/mpa-terrestrial/processed/tiles/part1.json
Allocating 32 GB of heap memory
[clean] Retained 2,922 of 2,922 features
[simplify] Repaired 464 intersections
[clean] Retained 2,922 of 2,922 features
[o] Wrote /home/mambauser/data/mpa-terrestrial/processed/tiles/part10.json
Allocating 32 GB of heap memory
[clean] Retained 2,923 of 2,923 features
[simplify] Repaired 434 intersections
[clean] Retained 2,923 of 2,923 features
[o] Wrote /home/mambauser/data/mpa-terrestrial/processed/tiles/part11.json
Allocat

In [31]:
mbtileGeneration(Path("/home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.json"))

For layer 0, using name "mpaterrestrialtiles"
/home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.json:24380: Found ] at top level
/home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.json:12366: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
292259 features, 138020430 bytes of geometry, 6110864 bytes of separate metadata, 14481131 bytes of string pool
Choosing a maxzoom of -z5 for features about 13778 feet (4200 meters) apart
Choosing a maxzoom of -z8 for resolution of about 1485 feet (452 meters) within features
tile 0/0/0 size is 845483 with detail 12, >500000    
tile 1/0/0 size is 528833 with detail 12, >500000    
tile 1/1/0 size is 972990 with detail 12, >500000    
tile 2/1/1 size is 614654 with detail 12, >500000    
tile 2/2/1 size is 1540068 with detail 12, >500000    
tile 2/2/1 size is 806964 with detail 11, >500000    
tile 3/4/2 size is 2272518 with detail 12, >500000 

PosixPath('/home/mambauser/data/mpa-terrestrial/processed/tiles/mpa-terrestrial-tiles.mbtiles')

In [ ]:
pipe = "mpa-terrestrial"
collection_name = "pas_wdpa"

source_dir = FileConventionHandler(pipe)

# Download the EEZ file && unzip it
download_and_unzip_if_needed(source_dir, prev_step, mysettings, "gpkg")
# split the file in two parts
file = gpd.read_file(source_dir.get_step_fmt_file_path(prev_step, "shp").as_posix())
i_folder = source_dir.get_step_path(prev_step)
folders = split_2_parts(file, i_folder)
# simplify the geometries
Mapshaper(16).input(folders).filter_fields(fields="WDPAID,NAME,PA_DEF,GIS_AREA,PARENT_ISO").clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).merge_layers().output(
    source_dir.get_step_fmt_file_path(current_step, "json").as_posix(), force=True, format="geojson"
).execute()

# generate the mbtiles
mbtileGeneration(source_dir.get_step_fmt_file_path(current_step, "json"))

/home/mambauser/data/mpa/processed/mpa_preprocess.zip
/home/mambauser/data/mpa/processed/preprocess


Allocating 16 GB of heap memory
[clean] Retained 19,184 of 19,184 features
[simplify] Repaired 277,407 intersections; 443,678 intersections could not be repaired
[clean] Retained 19,184 of 19,184 features
[o] Wrote /home/mambauser/data/mpa/processed/tiles/mpa_tiles.json


PosixPath('/home/mambauser/data/mpa/processed/tiles/mpa_tiles.mbtiles')

In [ ]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpa/processed/tiles/mpa_tiles.mbtiles to s3://tilestream-tilesets-production/de/_pending/yvng0dxxxru12eq9ye80350mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


True

### MPA Atlas 

In [7]:
pipe = "mpaatlas"
collection_name = "mpa_atlas"

source_dir = FileConventionHandler(pipe)
# Download the data file && unzip it if needed
download_and_unzip_if_needed(source_dir, prev_step, mysettings)

# generate the mbtiles
mbtileGeneration(
    source_dir.get_step_fmt_file_path(prev_step, "shp"),
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
)

/home/mambauser/data/mpaatlas/processed/mpaatlas_preprocess.zip
/home/mambauser/data/mpaatlas/processed/preprocess


Allocating 50 GB of heap memory
[clean] Retained 863 of 870 features
[o] Wrote /home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json
For layer 0, using name "mpaatlas_preprocess"
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:293: Reached EOF without all containers being closed
In JSON object {"type":"FeatureCollection","features":[]}
/home/mambauser/data/mpaatlas/processed/preprocess/mpaatlas_preprocess.json:21: Found ] at top level
863 features, 33449716 bytes of geometry, 29354 bytes of separate metadata, 53170 bytes of string pool
Choosing a maxzoom of -z0 for features about 282844 feet (86211 meters) apart
Choosing a maxzoom of -z12 for resolution of about 98 feet (30 meters) within features
  99.9%  12/1004/2052  


PosixPath('/home/mambauser/data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles')

In [8]:
uploadToMapbox(
    source_dir.get_step_fmt_file_path(current_step, "mbtiles"),
    collection_name,
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

upload: ../../data/mpaatlas/processed/tiles/mpaatlas_tiles.mbtiles to s3://tilestream-tilesets-production/32/_pending/4py3xz71znm1h1p9cr1v050mc/skytruth


Linking tileset to Mapbox: 100%|██████████| 100/100 [05:24<00:00,  3.25s/it]


True

### Protected seas

In [37]:
pipe = "protectedseas"
collection_name = "protected_seas"

pipe_dir = FileConventionHandler(pipe)
input_file = pipe_dir.get_processed_step_path(prev_step).joinpath("ProtectedSeas_20240716.geojson")
output_file = pipe_dir.get_processed_step_path(current_step).joinpath("protectedseas_tiles.mbtiles")


# Download the protected seas file && unzip it
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name="ProtectedSeas/ProtectedSeas_20240716.geojson",
    file=input_file,
    operation="r",
)

# Load the data
protectedseas_layer = gpd.read_file(input_file)

In [38]:
# simplify the geometries
Mapshaper(16).input([input_file.as_posix()]).clean(
    allow_overlaps=True, rewind=True
).simplify("dp 10% keep-shapes planar").clean(allow_overlaps=True).output(
    input_file.as_posix(),
    force=True,
    format="geojson",
).execute()

Allocating 16 GB of heap memory
[clean] Retained 6,741 of 6,741 features
[simplify] Repaired 5,240 intersections; 629 intersections could not be repaired
[clean] Retained 6,741 of 6,741 features
[o] Wrote /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson


CompletedProcess(args='mapshaper-xl 16gb -i /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson  -clean allow-overlaps rewind -simplify dp 10% keep-shapes planar -clean allow-overlaps -o /home/mambauser/data/protectedseas/processed/preprocess/ProtectedSeas_20240716.geojson force format=geojson', returncode=0)

In [39]:
mbtileGeneration(input_file, output_file)

PosixPath('/home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles')

In [41]:
uploadToMapbox(
    output_file,
    "protected_seas",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

rosetta error: failed to open elf at /lib64/ld-linux-x86-64.so.2
 Trace/breakpoint trap


CalledProcessError: Command 'aws s3 cp /home/mambauser/data/protectedseas/processed/tiles/protectedseas_tiles.mbtiles s3://tilestream-tilesets-production/71/_pending/lxene85xrno1vgk2b6c6350mc/skytruth --region us-east-1' returned non-zero exit status 133.

### Habitat layers

In [ ]:
# TODO: Add all the habitats layer processing
habitat = mysettings.DATA_DIR.joinpath("habitat_intermediate", "layer").resolve()

#### Warm water corals

In [ ]:
warm_water_corals = habitat.joinpath("warm_water_corals.mbtiles").resolve()

In [ ]:
uploadToMapbox(
    warm_water_corals,
    "warm_water_corals",
    mysettings.MAPBOX_USER,
    mysettings.MAPBOX_TOKEN,
)

### Contextual layers

In [ ]:
# Todo: we need to add the contextual layers upload here